###### Nipype Workflow for ProbTrackX2 fdt_path waypath thresholding and normalization
## Version 9: Threshold then normalize

In [1]:
from nipype import config
cfg = dict(execution={'remove_unnecessary_outputs': False,
                      'keep_inputs': True},
           monitoring={'enabled': True,
                       'sample_frequency': 5,
                       'summary_file': './resource_monitor.json'
                      }
          )
config.update_config(cfg)

In [19]:
from os.path import abspath
from IPython.display import Image

from nipype import SelectFiles, Node, MapNode, Workflow, Function
from nipype.interfaces import fsl
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import ImageStats
from nipype.interfaces.io import DataSink
from nipype.interfaces.utility import IdentityInterface

import networkx
import graphviz as gv
import os
import base64
import re
import glob
import sys
import subprocess

import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import copy

## Select Files:

In [3]:
# Node: SelectFiles
## Selects subject folders from "/data/NipypeScratch/runpbx2/"
#s_templates={'subject': '_subject_id_100[2-3]*'} # 2 subjects
s_templates={ 'fdt_paths': '_subject_id_*/_seed*/pbx2/fdt_paths.nii.gz' } 

sf0 = Node(SelectFiles(s_templates),
          name='selectfiles0'
          )
sf0.inputs.base_directory = '/data/NipypeScratch/runpbx2/'

# Save fdt_paths list as variable 'fdt+paths'
sf0_results= sf0.run().outputs
fdt_paths= sf0_results.fdt_paths

180730-23:13:34,339 workflow INFO:
	 [Node] Setting-up "selectfiles0" in "/tmp/tmp96xwwpgu/selectfiles0".
180730-23:13:34,343 workflow INFO:
	 [Node] Running "selectfiles0" ("nipype.interfaces.io.SelectFiles")
180730-23:13:36,351 workflow INFO:
	 [Node] Finished "selectfiles0".


In [4]:
infosource = Node(IdentityInterface(fields=['fdt_paths']),
                  name="infosource")
infosource.iterables = [ ('fdt_paths', fdt_paths) ]

## Normalize fdt_paths using waytotal:

In [5]:
# Node: Function: Normalizes using FSL Maths -div
# ifiles: original input file
Normalize = Node(fsl.BinaryMaths(operation='div'),
                 name = 'Normalize',
                )
Normalize.inputs.output_type= 'NIFTI_GZ'
# Normalize.in_file= # From pipeline: fdt_paths
# Normalize.inputs.operand_value= # From pipeline: waytotal

## Threshold fdt_paths using waytotal, Should not account for waytotal if already normalized:

In [6]:
# Node: Function: Thresholds using FSL Maths
# ifiles: original input file
Threshold = Node(fsl.Threshold(),
                 name = 'Threshold',
                )
#Threshold.inputs.thresh= 1 # Normalized threshold is 1
Threshold.output_type= 'NIFTI_GZ'
#Threshold.use_robust_range= True

#Threshold.in_file= # From pipeline: fdt_paths
#Threshold.thresh= # From pipeline: waytotal/or constant
#Threshold.out_file= # From pipeline: OPTIONAL

## Organize thresholded niftis in /SASRAID/ThreshNipype/output

In [7]:
Sink = Node(DataSink(), name = 'Sink')
Sink.inputs.base_directory= '/data/SASRAID/ThreshNipype/output'
Sink.inputs.substitutions= [('_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_', '')
                            ('.._seed_..data..HCP_Data..EHECHT_ROIS..', '')]

## Initialize Workflows:

In [9]:
# Workflow: Initialize
wf = Workflow(name="ThreshNipype")
wf.base_dir = '/data/SASRAID/'

## Connect Nodes:

In [10]:
def getWaytotal(x, Tscaling=10000): 
    import subprocess
    import re
    
    x= re.sub('fdt_paths.nii.gz', 'waytotal', x)
    mycommand= "cat "+str(x)
    mycommand= mycommand.split(sep=' ')
    result= subprocess.run(mycommand, stdout=subprocess.PIPE)
    tmpwaytotal= result.stdout.decode("utf-8").split(' ')[0]
    #print('waytotal: %s' % tmpwaytotal)
    
    if(not tmpwaytotal): 
        print('Waytotal Empty for: %s' % pathwaytotal)
    else:
        waytotals= float(tmpwaytotal) / Tscaling                
        return( waytotals )

In [11]:
def cleanOutput(x):
    import re
    
    x= re.search('_id_(.*?).._seed_..data.*EHECHT_ROIS..(.*?)_(.*?)_(.*?).nii.gz', x)
    x= '%s_%s_%s_%s' % (x.group(1), x.group(2), x.group(3), x.group(4))
    return(x)

In [12]:
# Connect Nodes
wf.connect(infosource, "fdt_paths", Threshold, "in_file")
wf.connect(infosource, ("fdt_paths", getWaytotal, 10000), Threshold, "thresh")

wf.connect(Threshold, "out_file", Normalize, "in_file")
wf.connect(infosource, ("fdt_paths", getWaytotal, 1), Normalize, "operand_value")

wf.connect(Threshold, ("out_file", cleanOutput), Sink, "base_directory")
wf.connect(Threshold, "out_file", Sink, "container")

## Run Workflow:

In [15]:
# Run Workflow
#wf.run()
wf.run(plugin='MultiProc', plugin_args={'n_procs': 20})

180731-00:41:03,847 workflow INFO:
	 Workflow ThreshNipype settings: ['check', 'execution', 'logging', 'monitoring']
180731-00:44:28,886 workflow INFO:
	 Running in parallel.
180731-00:44:29,497 workflow INFO:
	 [MultiProc] Running 0 tasks, and 2760 jobs ready. Free memory (GB): 169.98/169.98, Free processors: 20/20.
180731-00:44:29,731 workflow INFO:
	 [Job 0] Cached (ThreshNipype.Threshold).
180731-00:44:29,742 workflow INFO:
	 [Job 3] Cached (ThreshNipype.Threshold).
180731-00:44:29,752 workflow INFO:
	 [Job 6] Cached (ThreshNipype.Threshold).
180731-00:44:29,762 workflow INFO:
	 [Job 9] Cached (ThreshNipype.Threshold).
180731-00:44:29,772 workflow INFO:
	 [Job 12] Cached (ThreshNipype.Threshold).
180731-00:44:29,782 workflow INFO:
	 [Job 15] Cached (ThreshNipype.Threshold).
180731-00:44:29,793 workflow INFO:
	 [Job 18] Cached (ThreshNipype.Threshold).
180731-00:44:29,803 workflow INFO:
	 [Job 21] Cached (ThreshNipype.Threshold).
180731-00:44:29,813 workflow INFO:
	 [Job 24] Cached 

	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_898176.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:34,82 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")
180731-00:44:34,86 workflow INFO:
	 [Job 35] Cached (ThreshNipype.Normalize).
180731-00:44:34,86 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_898176.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Sink".180731-00:44:34,89 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".


180731-00:44:34,97 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")180731-00:44:34,102 workflow INFO:
	 [Job 38] Cached (ThreshNipype.Normalize).
180731-00:44:34,104 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".180731-0

180731-00:44:37,821 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_896778.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Right.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:37,830 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")

180731-00:44:37,837 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".180731-00:44:37,841 workflow INFO:
	 [Job 68] Cached (ThreshNipype.Normalize).
180731-00:44:37,840 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_896778.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:37,847 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")

180731-00:44:37,853 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".180731-00:44:37,857 workflow INFO:
	 [Job 71] 

	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_894673.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:40,168 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")
180731-00:44:40,175 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".180731-00:44:40,178 workflow INFO:
	 [Job 110] Cached (ThreshNipype.Normalize).

180731-00:44:40,178 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_894067.._seed_..data..HCP_Data..EHECHT_ROIS..Human_Hypothalamus_Right.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:40,185 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")
180731-00:44:40,191 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".180731-00:44:40,194 workflow INFO:
	 [Job 113] Cached (ThreshNipype.Normalize).

1

	 [Node] Finished "ThreshNipype.Sink".
180731-00:44:43,850 workflow INFO:
	 [Job 140] Cached (ThreshNipype.Normalize).
180731-00:44:43,849 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_891667.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:43,856 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")
180731-00:44:43,861 workflow INFO:
	 [Node] Finished "ThreshNipype.Sink".
180731-00:44:43,866 workflow INFO:
	 [Job 143] Cached (ThreshNipype.Normalize).
180731-00:44:43,865 workflow INFO:
	 [Node] Setting-up "ThreshNipype.Sink" in "/data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_891667.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Sink".
180731-00:44:43,872 workflow INFO:
	 [Node] Running "Sink" ("nipype.interfaces.io.DataSink")


...

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing

  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/pyth

180731-00:54:24,670 workflow INFO:
	 [Job 4323] Cached (ThreshNipype.Threshold).

  File "/opt/conda/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


# Merge Males/Females

In [138]:
metaloc= '/code/notebooks/postPBX/unrestricted_dagutman_7_12_2018_15_39_53.csv'
metadata= pd.read_csv(metaloc,header=0)

In [139]:
bedpostx_dir= '/data/SASRAID/ThreshNipype/_fdt_paths*/Normalize/'
bedpostx_results= [ fn for fn in glob.glob(bedpostx_dir) ]

subjectinfo= {}
for i in bedpostx_results:
    output_sid= re.search('.*_subject_id_(.*?)\.\..*', i).group(1)
    fdt_paths= glob.glob( i+'fdt_paths*.nii.gz' )
    subjectinfo[output_sid]= fdt_paths[0]

In [140]:
# Copy original metadata
finalmetadata= copy.deepcopy(metadata)

In [141]:
# Set FDT_Paths column
filler= [''] * metadata.shape[0]
for k,v in subjectinfo.items():
    k= int(k)
    idx_row= int( np.where(finalmetadata['Subject'] == k)[0][0] )
    filler[idx_row]= v
finalmetadata['FDT_Paths']= filler

In [142]:
gender_info= finalmetadata.groupby(by='Gender')

In [143]:
# Subject ID grouped by gender:
Females= list( gender_info.get_group('F')['FDT_Paths'] )
Males= list( gender_info.get_group('M')['FDT_Paths'] )

In [146]:
females= [ i for i in Females if len(i) > 0]
males= [ i for i in Males if len(i) > 0]

## Identity: Female/Male

In [149]:
infosourceG = Node(IdentityInterface(fields=['Female','Male']),
                  name="infosourceG")
infosourceG.iterables = [ ('Female', females), ('Male', males) ]

## Select Files

In [147]:
# Node: SelectFiles
templates={'Males': males,
           'Females': females,
          } 

sfG = Node(SelectFiles( templates ),
          name='selectfilesG'
          )
sfG.inputs.base_directory = '/'

# Save fdt_paths list as variable 'fdt+paths'
sfG.run().outputs

TypeError: expected str, got list

## Merge Niftis

In [ ]:
from nipype.interfaces.fsl import Merge

merger = Node(Merge(),
              name = 'Merger'
             )
merger.inputs.dimension= 't'
merger.inputs.output_type= 'NIFTI_GZ'
merger.inputs.merged_file= '/data/SASRAID/ThreshNipype/GenderDiff/Females'
merger.inputs.in_files= females
merger.run().outputs

180731-02:19:27,64 workflow INFO:
	 [Node] Setting-up "Merger" in "/tmp/tmp5ip_fe_3/Merger".
180731-02:19:27,98 workflow INFO:
	 [Node] Running "Merger" ("nipype.interfaces.fsl.utils.Merge"), a CommandLine Interface with command:
fslmerge -t /data/SASRAID/ThreshNipype/GenderDiff/Females /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_100307.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Left.nii.gz..pbx2..fdt_paths.nii.gz/Normalize/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_101006.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Normalize/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/_fdt_paths_..data..NipypeScratch..runpbx2.._subject_id_101915.._seed_..data..HCP_Data..EHECHT_ROIS..Human_BasalForebrain_Bilat.nii.gz..pbx2..fdt_paths.nii.gz/Normalize/fdt_paths_thresh_maths.nii.gz /data/SASRAID/ThreshNipype/_fdt_paths_

In [152]:
# Connect Nodes
wf.connect(males, merger, "in_files")
wf.connect(females, "Females", merger, "in_files")

TypeError: connect() takes either 4 arguments, or 1 list of connection tuples (3 args given)